In [1]:
using GLMakie
using JLD

In [2]:
function load_matrix(timestep::Int, file_name::String, var::String)
    data_name = var * "_$timestep"
    c = jldopen(file_name, "r") do file
        return read(file, data_name)
    end
end

load_matrix (generic function with 1 method)

In [3]:
function get_metadata(file_name::String)
    c = jldopen(file_name, "r") do file
        return read(file, "metadata")
    end
end

get_metadata (generic function with 1 method)

In [39]:
function create_gif(file_name::String)
    create_gif(file_name, file_name)
end

function create_gif(file_name::String, animation_file_name::String)
    metadata = get_metadata(file_name)
    println(metadata)

    T = floor(Int, metadata["T"])
    dt = metadata["dt"]
    sample_timestep = 2*metadata["sample_timestep"]
    sample_interval = metadata["sample_interval"]

    zeta = load_matrix(0, file_name, "zeta")
    psi = load_matrix(0, file_name, "psi")
    shift_amounts = (M/2, P/2)

    zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
    zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts)

    zeta_top_plot = Observable(zeta_top_shifted)
    zeta_bottom_plot = Observable(zeta_bottom_shifted)

    psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
    psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts)

    psi_top_plot = Observable(psi_top_shifted)
    psi_bottom_plot = Observable(psi_bottom_shifted)

    fig = Figure(size = (900, 600))
    ga = fig[1, 1] = GridLayout()
    
    ax_top_left = Axis(ga[1, 1], title="Zeta Layer 1")
    ax_bottom_left = Axis(ga[2, 1], title="Zeta Layer 2")
    ax_top_right = Axis(ga[1, 2], title="Psi Layer 1")
    ax_bottom_right = Axis(ga[2, 2], title="Psi Layer 2")

    heatmap!(ax_top_left, zeta_top_plot)
    heatmap!(ax_bottom_left, zeta_bottom_plot)
    heatmap!(ax_top_right, psi_top_plot)
    heatmap!(ax_bottom_right, psi_bottom_plot)

    # animation_file_name = file_name * "_heatmap.mp4"

    total_steps = floor(Int, T / dt)

    println("Saving .mp4 to $animation_file_name")

    framerate = 10

    record(fig, animation_file_name, 0:sample_timestep:total_steps, framerate = framerate) do timestep
        zeta = load_matrix(Int(timestep), file_name, "zeta")
        psi = load_matrix(Int(timestep), file_name, "psi")

        zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
        zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts) 

        zeta_top_plot[] = zeta_top_shifted
        zeta_bottom_plot[] = zeta_bottom_shifted

        psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
        psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts) 

        psi_top_plot[] = psi_top_shifted
        psi_bottom_plot[] = psi_bottom_shifted
    end
end

function show_animation(file_name::String)
    metadata = get_metadata(file_name)
    println(metadata)

    T = floor(Int, metadata["T"])
    dt = metadata["dt"]
    sample_timestep = metadata["sample_timestep"]
    sample_interval = metadata["sample_interval"]

    zeta = load_matrix(0, file_name, "zeta")
    psi = load_matrix(0, file_name, "psi")

    # shift_amounts = (256, 128)
    shift_amounts = (M/2, P/2)

    zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
    zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts)

    zeta_top_plot = Observable(zeta_top_shifted)
    zeta_bottom_plot = Observable(zeta_bottom_shifted)

    psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
    psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts)

    psi_top_plot = Observable(psi_top_shifted)
    psi_bottom_plot = Observable(psi_bottom_shifted)

    # plot_time = Observable(0.0)

    fig = Figure(size = (900, 600))
    ga = fig[1, 1] = GridLayout()
    
    ax_top_left = Axis(ga[1, 1], title="Zeta Layer 1")
    ax_bottom_left = Axis(ga[2, 1], title="Zeta Layer 2")
    ax_top_right = Axis(ga[1, 2], title="Psi Layer 1")
    ax_bottom_right = Axis(ga[2, 2], title="Psi Layer 2")

    heatmap!(ax_top_left, zeta_top_plot)
    heatmap!(ax_bottom_left, zeta_bottom_plot)
    heatmap!(ax_top_right, psi_top_plot)
    heatmap!(ax_bottom_right, psi_bottom_plot)

    expected_zeta_0 = (-1 / (4pi^2 * model.Lx^-2 + 32pi^2Ly^-2)) * psi[:,:,1]

    # display(psi_top_shifted)

    display(fig)

    # display(psi[:,:,1])
    # display(psi[:,:,2])

    fps = 10
    sleep_time = 1 / fps

    for (timestep, time) in enumerate(0:dt:T)
        if timestep % (sample_timestep) == 0
            zeta = load_matrix(Int(timestep), file_name, "zeta")
            psi = load_matrix(Int(timestep), file_name, "psi")

            println("$file_name, $timestep, zeta top | ", zeta[10,10,1])
            println("$file_name, $timestep, zeta bottom | ", zeta[10,10,2])
            println("$file_name, $timestep, psi top | ", psi[10,10,1])
            println("$file_name, $timestep, psi bottom | ", psi[10,10,2])
            # println("$file_name, $timestep, bottom | ", zeta[10,10,2])

            zeta_top_shifted = circshift(zeta[:,:,1], shift_amounts)
            zeta_bottom_shifted = circshift(zeta[:,:,2], shift_amounts) 

            zeta_top_plot[] = zeta_top_shifted
            zeta_bottom_plot[] = zeta_bottom_shifted

            psi_top_shifted = circshift(psi[:,:,1], shift_amounts)
            psi_bottom_shifted = circshift(psi[:,:,2], shift_amounts) 

            psi_top_plot[] = psi_top_shifted
            psi_bottom_plot[] = psi_bottom_shifted

            display(fig)

            # Set the matrix to nothing so the memoery can be freed up later by the garbage collector.
            # matrix = nothing
            
            # # Force garbage collection to clean up the memory used by the old array.
            # GC.gc()
            sleep(sleep_time)
        end
    end

    sleep(1)
end

show_animation (generic function with 1 method)

In [60]:
include("../../src/run_model.jl")

H_1 = 1.0*KM
H_2 = 2.0*KM
beta = 2*10^-11
Lx = 4000.0*KM # 4000 km
Ly = 4000.0*KM # 2000 km
dt = 5.0*MINUTES # 30 minutes
T = 0.5*YEAR  # Expect to wait 90 days before seeing things.
U = 3.0 # Forcing term of top level.
M = P =128
dx = Lx / M
# P = Int(Ly / dx)
visc = 100.0 # Viscosity, 100m^2s^-1
r = 10^-7 # bottom friction scaler.
R_d = 60.0*KM # Deformation radious, ~40km. Using 60km for better numerics.

model = BaroclinicModel(H_1, H_2, beta, Lx, Ly, dt, T, U, M, P, dx, visc, r, R_d)

sim_name = "test_31"
data_file_name = "../../data/$sim_name.jld"

zeta, psi = run_model(model, data_file_name)

# expected_zeta_0 = (-1 / (4pi^2 * model.Lx^-2 + 32pi^2Ly^-2)) * psi[:,:,1,1]

Parameters:
(f_0^2 / N^2): 0.0001388888888888889
S1 = 1.3888888888888888e-10
S2 = 1.3888888888888888e-10
Beta_1 = 4.3666666666666666e-10
Beta_2 = -3.966666666666667e-10
M = 64
P = 64
dt = 300.0
T = 7.884e6
Total steps = 26280

Time to init Poisson system: 0.001002 seconds (764 allocations: 2.470 MiB)
Time to init modified Helmholtz system: 0.000558 seconds (721 allocations: 2.592 MiB)
Starting timeloop
Progress: 4.0 %
Progress: 8.0 %
Progress: 12.0 %
Progress: 16.0 %
Progress: 20.0 %
Progress: 24.0 %
Progress: 28.0 %
Progress: 32.0 %
Progress: 36.0 %
Progress: 40.0 %
Progress: 44.0 %
Progress: 48.0 %
Progress: 52.0 %
Progress: 56.0 %
Progress: 60.0 %
Progress: 64.0 %
Progress: 68.0 %
Progress: 72.0 %
Progress: 76.0 %
Progress: 80.0 %
Progress: 84.0 %
Progress: 88.0 %
Progress: 92.0 %
Progress: 96.0 %
Progress: 100.0 %


([3.7491484959378855e-8 -7.027459790197847e-7 … 3.7491484959378855e-8 -7.027459790197847e-7; 4.905144655764352e-7 7.541303547490558e-7 … 4.905144655764352e-7 7.541303547490558e-7; … ; 3.7491484959378855e-8 -7.027459790197847e-7 … 3.7491484959378855e-8 -7.027459790197847e-7; 4.905144655764352e-7 7.541303547490558e-7 … 4.905144655764352e-7 7.541303547490558e-7;;; 2.0798079250078676e-7 -4.028821508967069e-8 … 2.0798079250078676e-7 -4.028821508967069e-8; -2.523382072853607e-7 2.2255672275058853e-7 … -2.523382072853607e-7 2.2255672275058853e-7; … ; 2.0798079250078676e-7 -4.028821508967069e-8 … 2.0798079250078676e-7 -4.028821508967069e-8; -2.523382072853607e-7 2.2255672275058853e-7 … -2.523382072853607e-7 2.2255672275058853e-7;;;; 4.645252109899639e-8 -7.013404387459622e-7 … 4.645252109899639e-8 -7.013404387459622e-7; 4.866968361974061e-7 7.591942523657548e-7 … 4.866968361974061e-7 7.591942523657548e-7; … ; 4.645252109899639e-8 -7.013404387459622e-7 … 4.645252109899639e-8 -7.013404387459622e

In [28]:
create_gif(data_file_name, "full_model_heatmap.mp4")

Dict{String, Real}("T" => 3.1536e7, "sample_timestep" => 288, "dt" => 300.0, "sample_interval" => 86400.0)
Saving .mp4 to advection_only_heatmap.mp4


"advection_only_heatmap.mp4"

In [61]:
show_animation(data_file_name)

Dict{String, Real}("T" => 7.884e6, "sample_timestep" => 288, "dt" => 300.0, "sample_interval" => 86400.0)
../../data/test_31.jld, 288, zeta top | 1.606592715514876e-8
../../data/test_31.jld, 288, zeta bottom | -3.2629956618808247e-7
../../data/test_31.jld, 288, psi top | 103.26455675514147
../../data/test_31.jld, 288, psi bottom | 322.2311348455885
../../data/test_31.jld, 576, zeta top | 1.564436426383213e-8
../../data/test_31.jld, 576, zeta bottom | -3.235809075060187e-7
../../data/test_31.jld, 576, psi top | -415.77983160786755
../../data/test_31.jld, 576, psi bottom | -129.95869211707313
../../data/test_31.jld, 864, zeta top | -4.31904693285343e-7
../../data/test_31.jld, 864, zeta bottom | -3.2077763182116236e-7
../../data/test_31.jld, 864, psi top | 71.37037568374302
../../data/test_31.jld, 864, psi bottom | 75.34694841279205
../../data/test_31.jld, 1152, zeta top | -3.5229292870947085e-7
../../data/test_31.jld, 1152, zeta bottom | -3.1831743218517486e-7
../../data/test_31.jld, 115

In [59]:
include("../../src/schemes/arakawa.jl")

Lx = 10
Ly = 10
A_func(x, y) = sin(2pi*x/Lx)*sin(2pi*y/Ly)
B_func(x, y) = cos(2pi*x/Lx)*cos(2pi*y/Ly) 
true_J_func(x, y) = -4pi^2/(Lx*Ly)*cos(2pi*x/Lx)^2*sin(2pi*y/Ly)^2 + 4pi^2/(Lx*Ly)*sin(2pi*x/Lx)^2*cos(2pi*y/Ly)^2 


M = 64

dx = Lx / M

xs = 1:dx:Lx
ys = 1:dx:Ly
A = Matrix{Float64}(inflate(A_func, xs, ys))
B = Matrix{Float64}(inflate(B_func, xs, ys))

true_J_matrix = inflate(true_J_func, xs, ys)
test_J_matrix = J(dx, A, B)

display(test_J_matrix[1:4, 1:4])
display(true_J_matrix[1:4, 1:4])

error = dx * norm(test_J_matrix - true_J_matrix, Inf)
println(error)

plot(xs, ys, [u, u_true], st=:surface, layout=(1, 2), size=(1000, 400))

3×3 reshape(::UnitRange{Int64}, 3, 3) with eltype Int64:
 1  4  7
 2  5  8
 3  6  9

1
2
3
4
5
6
7
8
9
